# Càlcul de la signatura d'un conjunt de dades


Implementarem una classe Data per guardar les dades referents a la sèrie temporal de la qual en volem calcular la signatura. Els seus paràmetres són:
- times: un llistat de temps discrets on la sèrie temporal pren valors
- values: els valors de la sèrie temporal
- delta_X: una matriu amb les diferències entre valors de $X$ (self.values). Per índexos n,i, delta_X[n][i] guarda la diferència $X^i_{t_n} - X^i_{t_{n-1}}$
- d: la cardinalitat de l'alfabet (l'alfabet no ampliat, sense signes)

In [1]:
class Data: # Data class
    def __init__(self):
        self.times = [] # list of times
        self.values = [] # list of values
        
        

    # Calculate delta_X
    def calculate_delta_X(self):
        self.delta_X = [[0 for i in range(self.d)] for j in range(len(self.times)-1)]
        for i in range(len(self.times)-1): # iterate through the times
            for j in range(self.d): # iterate through the dimensions of the data
                self.delta_X[i][j] = self.values[i+1][j] - self.values[i][j] # calculate delta_X as the difference between the next and current value
    
    def set_times(self, times):
        self.times = times
    
    def set_values(self, values):
        self.values = values
        self.d = len(self.values[0])
    

Per guardar els indexos de cada element de la signatura, implementem la classe Words que guardarà totes les paraules de l'alfabet $\{0,...,d-1\}$. Per aquesta raó, aquesta implementació de la signatura tindrà un nombre finit d'elements.

In [2]:
class Words:
    # Words class stores a list of all words with length <= k
    def __init__(self, k, d):
        self.k = k  # k is the maximum length of the string
        self.combinations = []  # list of combinations
        self.d = d
        self.generate_combinations()  # generate all combinations of length <= k

    def generate_combinations(self):  # generate all combinations of numbers 0,..,d-1 with signs + and - of length <= k
        # Define a recursive function to generate combinations
        def generate_helper(prefix, length):
            if length == 0:
                self.combinations.append(prefix)
            else:
                for i in range(self.d):
                    generate_helper(prefix + str(i) + '+', length - 1)
                    generate_helper(prefix + str(i) + '-', length - 1)

        # Generate combinations of lengths from 0 to k
        for length in range(self.k + 1):
            generate_helper('', length)



In [3]:
# Example
words = Words(2,2)
print(words.combinations)
print('Number of combinations: ' ,len(words.combinations)) 

['', '0+', '0-', '1+', '1-', '0+0+', '0+0-', '0+1+', '0+1-', '0-0+', '0-0-', '0-1+', '0-1-', '1+0+', '1+0-', '1+1+', '1+1-', '1-0+', '1-0-', '1-1+', '1-1-']
Number of combinations:  21


La classe Signature implementarà el càlcul de la signatura discreta d'un conjunt de dades per paraules de mida <=k. Farem servir un algoritme de càlcul diferent a l'algoritme naive que simplement calcula de forma recursiva la signatura a partir de les equacions de la signatura discreta.
L'algoritme que farem servir segueix sent un algoritme recursiu que que calcula la signatura a partir de les equacions però desant els elements de la signatura ja calculats, en un diccionari, per no haver-los de recalcular.

És important puntualitzar que els indexos comencen a partir del 0, a diferència de la notació habitual que comencen a partir de 1.

Els seus paràmetres són:
- k: mida màxima de la paraula
- data: objecte de tipus Data
- mu: constant de decadència
- delta_mu: llista que, per un índex n, desa el valor de $e^{-\mu(t_n - t_{n-1})}$
- is_computed: diccionari que per una paraula w i uns n,m desa si ha estat calculat o no el valor de la signatura
- words: objecte de tipus Words
- sig: diccionari que per una paraula w i uns n,m desa el valor de la signatura



In [4]:
import math # import math library
class Signature:
    def __init__(self, k, data):
        self.k = k # maximum length of the word
        self.data = data # list of strings
        self.mu = 1 # rate of decay
        self.delta_mu = [] # list of exp(-mu*(t_n - t_{n-1}))
        self.iscomputed = {} # dictionary to store the computed values
        self.words = Words(self.k, self.data.d) # list of all words with length <= k
        # Initialize the dictionary iscomputed with False for all words
        for word in self.words.combinations: # iterate through the list of words
            for m in range(len(data.times)):
                for n in range(len(data.times)):
                    self.iscomputed[(word, m, n)] = False
        
        self.sig = {} # dictionary to store the signature values
        self.initialize_signature()
        
    def set_mu(self, mu): # set the rate of decay
        self.mu = mu
        
    def set_delta_mu(self): # set delta_mu
        self.delta_mu = [math.exp(-self.mu * (self.data.times[n] - self.data.times[n-1])) for n in range(1, len(self.data.times))] # exp(-mu*(t_n - t_{n-1}))
   

    def calculate_signature(self,t_m, t_n):
        m = self.data.times.index(t_m) # index of t_m
        n = self.data.times.index(t_n) # index of t_n
        output_sig = {}
        for word in self.words.combinations: # iterate through the list of words
            self.sig[word][m][n] = self.signature(m, n, word) # calculate the signature value
            output_sig[word] = self.sig[word][m][n] # store the signature value in the output_sig dictionary
        return output_sig # return the output_sig dictionary
    
    def initialize_signature(self): # initialize the signature values
        for word in self.words.combinations:
            self.sig[word] = [[0 for i in range(len(self.data.times))] for j in range(len(self.data.times))]
            

    def signature(self, m, n, word): # calculate an element of the signature
        if self.iscomputed[(word, m, n)]: # if the value is already computed
            return self.sig[word][m][n] # return the value
        
        if len(word) == 0: # if the length of the word is 0
            return 1 # return 1
        if m==n and len(word) != 1: # if the length of the word is >=1 and m=n
            return 0 # return 0
        #We use the recursive equation to calculate the signature
        w,i_ = word[:len(word)-2], word[len(word)-2:] # split the word into two parts
        i,sign = i_[0],i_[1] # split the second part into number and sign
        
        if sign == '-': # if it's head
            self.sig[word][m][n]  = self.delta_mu[n-1]*(self.signature(m, n-1, word) + self.data.delta_X[n-1][int(i)]*self.signature(m, n-1, w)) # calculate the signature value
        else: # if it's tail
            
            self.sig[word][m][n] =  self.delta_mu[n-1]*self.signature(m, n-1, word) + self.data.delta_X[n-1][int(i)]*self.signature(m, n, w) # calculate the signature value
        
        self.iscomputed[(word, m, n)] = True # set the value of iscomputed to True
    
        return self.sig[word][m][n] # return the signature value
    

In [11]:
# Example data object
import time
start_time = time.time()
data = Data()
times = [0, 1, 1.5, 2.5, 3]
values = [[1, 1], [3, 4], [3, 2], [5, 2], [8, 6]]
data.set_times(times)
data.set_values(values)

data.calculate_delta_X()

#Signature object
signature = Signature(2, data)
signature.set_mu(0.693)
signature.set_delta_mu()
print(signature.delta_mu)
# Calculate the signature
sig = signature.calculate_signature(0,3)
print("--- %s seconds ---" % (time.time() - start_time))
print(sig)

[0.5000735956957677, 0.7071588192872713, 0.5000735956957677, 0.7071588192872713]
{'': 1, '0+': 4.914464840798731, '0-': 3.0788497746331607, '1+': 4.042957896358364, '1-': 2.703653689615039, '0+0+': 19.57261860844203, '0+0-': 6.743688926844569, '0+1+': 20.15800656541911, '0+1-': 6.657564958746547, '0-0+': 11.65151677702903, '0-0-': 3.372340769900896, '0-1+': 12.565509508326071, '0-1-': 3.3292724474985307, '1+0+': 13.715231088464384, '1+0-': 0.21478948179181753, '1+1+': 18.336578202725576, '1+1-': -1.3286100212391103, '1-0+': 8.61132912315731, '1-0-': -0.6249079376702307, '1-1+': 12.190074777598674, '1-1-': -1.2502575795164685}
--- 0.0037789344787597656 seconds ---
